# ReInvent 2019 Amazon Personalize Workshop

![MainDiagram](static/imgs/image.png)

## Agenda

At a high level working with Amazon Personalize follows the steps in the diagram below:

![FlowDiagram](static/imgs/personalize_process.png)

The specific process that you will be following is:

1. Imports and Setup
1. Preparing Your Data
1. Importing Your Data
1. Selecting a Recipe
1. Training a Solution
1. Deploying a Campaign
1. Getting Recommendations
1. Real-Time Interactions
1. Conclusion
1. Bonus: Bulk Export of Recommendations

In each of these steps you'll see and execute code snippets written in Python using our Boto3 SDK, these snippets can be modified to be components of your production integration with Personalize. 

This notebook will walk you through the steps to build a recommendation model for movies that are tailored to specific users, the goal is to recommend movies based on that users' history of positive interactions with movies.

The data is provided via the [MovieLens Project](https://movielens.org), you can read more about it later if you are interested.

The content below has been written to guide you through the process within the timeline of a workshop session. You can deploy the CloudFormation template you found on GitHub later to your own account and can walk through notebooks 1-3 to do the same exercise again. Additionally you can update the content in the cells to reflect your own data and it is an effective path to building a custom recommendaiton model for your use case.

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` or `Shift` + `Return`(Mac Users) while in the cell to execute it and move onto the next one.


As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.



## Imports and Setup 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools. The cell below will import them for use here. It will also update the boto3 library to the latest version. You may see a warning(yellow text) or an error(red text), this is perfectly fine, just continue running the next cell.

In [3]:
# Update boto3
!pip install --upgrade boto3
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

Requirement already up-to-date: boto3 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.10.28)
Requirement not upgraded as not directly required: s3transfer<0.3.0,>=0.2.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from boto3) (0.2.1)
Requirement not upgraded as not directly required: botocore<1.14.0,>=1.13.28 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from boto3) (1.13.28)
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from boto3) (0.9.4)
Requirement not upgraded as not directly required: urllib3<1.26,>=1.20; python_version >= "3.4" in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from botocore<1.14.0,>=1.13.28->boto3) (1.23)
Requirement not upgraded as not directly required: python-dateutil<2.8.1,>=2.1; python_version >= "2.7" in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [4]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

## Preparing Your Data

To begin you will need a collection of data points where users have interacted with content in some way. Amazon Personalize assumes that if an interaction is recorded it is a positive one, you will see how we use that later. 

The cell below will download the data you need, extract the content from a Zip file, and then display a small portion of it to your screen.

In [6]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'], engine='python')
pd.set_option('display.max_rows', 5)
data

--2019-11-29 00:49:30--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-100k.zip’ not modified on server. Omitting download.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflat

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


As you can see the data contains a UserID, ItemID, Rating, and Timestamp.

We do not need the `Rating` column so it will be removed before we save the file.

Once the data has been prepared you will save it to a CSV file locally with the last line.

In [8]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-ml-100k-prepared.csv"
data.to_csv(filename, index=False)

Before you can upload the data into S3 you will need to create a bucket, the cell below will do that.

In [9]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventpersonalizeworkshop"
print(bucket_name)
s3.create_bucket(Bucket=bucket_name)

059124553121reinventpersonalizeworkshop


{'ResponseMetadata': {'RequestId': '6AF4232B49FAE9B2',
  'HostId': 'G52SzCnu9N3mody2YNOOK8EkVN+/1yRV2rs65BskFzWBOwWSViLHJpGkot1Fx8582IqA2TkGBUk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'G52SzCnu9N3mody2YNOOK8EkVN+/1yRV2rs65BskFzWBOwWSViLHJpGkot1Fx8582IqA2TkGBUk=',
   'x-amz-request-id': '6AF4232B49FAE9B2',
   'date': 'Fri, 29 Nov 2019 00:50:39 GMT',
   'location': '/059124553121reinventpersonalizeworkshop',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/059124553121reinventpersonalizeworkshop'}

Now you can upload the file to S3 and it will be ready to import into Amazon Personalize.

In [10]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## Importing Your Data

The steps for importing your data are:

1. Create a Dataset Group.
1. Determine the schema for your dataset.
1. Create a Dataset using your schema.
1. Run an ImportJob to load the data for use with Personalize.


In Amazon Personalize a Dataset Group is how your information is isolated from any other experiment. No information is shared between these groups at all. A Dataset Group can contain your interaction data, item metadata, user metadata, event trackers, solutions, and campaigns. To learn more about them: https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 

Begin by creating a Dataset Group

#### Create Dataset Group

In [11]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-RI-demo",
  "ResponseMetadata": {
    "RequestId": "d781f6c8-6845-43b6-bba4-e991479fbb76",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 00:50:46 GMT",
      "x-amzn-requestid": "d781f6c8-6845-43b6-bba4-e991479fbb76",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Dataset Group Create Loop

It takes a few seconds to create a DatasetGroup so the cell below will poll until the DatasetGroup is active and you can continue with the workshop. Once it says `ACTIVE` move to the `Create Schema` step.


In [13]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

DatasetGroup: ACTIVE


#### Create Schema


Now you can determine the Schema for your Dataset. In this workshop time is limited so you will only be utilizing user-item-interaction data or interaction data for short. The cell below contains the required items for this dataset and maps to the structure of the CSV that you uploaded earlier.

In [14]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

The cell below will create a schema inside Amazon Personalize that can be connected to a Dataset this is how Personalize understands the content within your CSV file.

In [15]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:059124553121:schema/personalize-ri-demo-schema",
  "ResponseMetadata": {
    "RequestId": "5c5d9f3b-b826-4bc9-9ed9-e1cfb55ebc59",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 00:52:37 GMT",
      "x-amzn-requestid": "5c5d9f3b-b826-4bc9-9ed9-e1cfb55ebc59",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Dataset


Next create the Dataset for interactions and assign it the Schema provided and assign it to the Dataset Group you created earlier with this code:

In [16]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:059124553121:dataset/personalize-RI-demo/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "194334f7-1363-404e-9c0b-8a3f7f4fff82",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 00:52:39 GMT",
      "x-amzn-requestid": "194334f7-1363-404e-9c0b-8a3f7f4fff82",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Attach Policy to S3 Bucket

Before you can execute the import job for your data you will need to attach a bucket policy to your S3 bucket to allow Personalize to communicate with it, as well as an IAM role for the service to use within this AWS account.

In [17]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'CFF77C150CEF952A',
  'HostId': 'bmCCX7U940Zc5GyR/u7QG5+GcKmgPhRA+LCeUEeGKQh2AgvdmroEFjT2zqt8W8Vzgv3KPPQlmMg=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'bmCCX7U940Zc5GyR/u7QG5+GcKmgPhRA+LCeUEeGKQh2AgvdmroEFjT2zqt8W8Vzgv3KPPQlmMg=',
   'x-amz-request-id': 'CFF77C150CEF952A',
   'date': 'Fri, 29 Nov 2019 00:52:46 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

Just a note, IAM changes in the cell below take just a few seconds to be confirmed so this section will take 1 minute to complete. Execute the cell below next.

#### Create Personalize Role

In [18]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::059124553121:role/PersonalizeRoleRIDemo


At last you are ready to import the data into Personalize, the first cell below will start the process and the second contains a while loop that will poll the service to determine when the import job has completed. 

During this workshop you are importing a relatively small dataset and it may seem like this takes a while for such a small file. The leading use of time here is provisioning dedicated resources that will actually run the task. This allows Personalize to provide HIPAA compliance for example, it also means that just because your files are larger that it won't take an extremely long time, once the resources are up and running the import is pretty quick.

It may take up to 20 minutes for the import process to complete. 

#### Create Dataset Import Job

In [19]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:059124553121:dataset-import-job/personalize-ri-import",
  "ResponseMetadata": {
    "RequestId": "d8c3927f-0149-4b94-b8e2-5b73fbcd0814",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 00:56:03 GMT",
      "x-amzn-requestid": "d8c3927f-0149-4b94-b8e2-5b73fbcd0814",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Dataset Import Loop

The cell below will take a bit longer to complete, it is polling to learn when your Dataset has been fully imported into Personalize. This should take around 20 minutes to complete. Most of the time is spent provisioning infrastructure behind the scenes. This means that much larger files do not take much longer to import in most cases.

Run the cell below, then move on when it reaches an `ACTIVE` state.


In [20]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  12:56:21 AM
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
Import Completed on:  01:12:23 AM


## Selecting a Recipe

Inside Personalize the algorithms available are called recipes, the code below will give you a full list of them. They each have various use cases and detailed information can be found here: https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

In this workshop you will be using HRNN(Hierarchical Recurrent Neural Network) quoting the docs:

```
HRRN is a hierarchical recurrent neural network, which can model the user-item interactions across a given timeframe. Use the HRNN recipe when user behavior changes over time, which is referred to the evolving intent problem.

To train a model, HRNN uses the Interactions dataset from a dataset group. A dataset group is a set of related datasets, which can include the Users, Items, and Interactions datasets.
```

A paper explaining it more in detail as it relates to recommendations: https://openreview.net/pdf?id=ByzxsrrkJ4 


It is used here as an example of a recommendation system built using deep neural networks and can be trained using only our interactions data. This is a great place to start when running your own experiments later as well.

In [21]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

The cell below selects the `HRNN` recipe.

In [22]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Training a Solution

Within Amazon Personalize a model trained on customer data is called a Solution, these are versioned and so you will first create a Solution, then a Solution Version. The versions are utilized to track model improvement over time based on newer available data. 

Creating a Solution itself is nearly instantaneous however the actual training to create a version can take a bit of time. This is usually the longest waiting period in the process. If you are doing this outside of a workshop it is a great time to check your emails, grab a coffee, etc.

#### Create Solution

In [23]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn",
  "ResponseMetadata": {
    "RequestId": "2aa1f40d-5815-4d98-8250-2e14d7db8666",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 01:21:14 GMT",
      "x-amzn-requestid": "2aa1f40d-5815-4d98-8250-2e14d7db8666",
      "content-length": "94",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [24]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn/da0717fb",
  "ResponseMetadata": {
    "RequestId": "a75a5b00-f530-42a3-bc55-9c2ee17e2751",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 01:21:16 GMT",
      "x-amzn-requestid": "a75a5b00-f530-42a3-bc55-9c2ee17e2751",
      "content-length": "110",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version Loop

Training a model can take a bit of time, this will often take at least 20 minutes to complete. Run the cell below, again waiting for it to reach an `ACTIVE` state before moving to the next bit.

In [25]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

Training Started on:  01:21:17 AM
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRES

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance.

In [26]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn/da0717fb",
  "metrics": {
    "coverage": 0.3445,
    "mean_reciprocal_rank_at_25": 0.0349,
    "normalized_discounted_cumulative_gain_at_10": 0.0551,
    "normalized_discounted_cumulative_gain_at_25": 0.071,
    "normalized_discounted_cumulative_gain_at_5": 0.0408,
    "precision_at_10": 0.0101,
    "precision_at_25": 0.0067,
    "precision_at_5": 0.0112
  },
  "ResponseMetadata": {
    "RequestId": "36453efa-ed36-488e-879f-4458c38b7dfc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 02:00:28 GMT",
      "x-amzn-requestid": "36453efa-ed36-488e-879f-4458c38b7dfc",
      "content-length": "407",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [27]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-ri-camp",
  "ResponseMetadata": {
    "RequestId": "c45094c9-db02-4edb-8b00-28c2390e1387",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 29 Nov 2019 02:00:29 GMT",
      "x-amzn-requestid": "c45094c9-db02-4edb-8b00-28c2390e1387",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

#### Create a Campaign Loop

In this section Personalize is deploying your model and that takes a few minutes to complete. Execute the cell below to agian poll until the task has completed. It should take 10 to 15 minutes. Once the cell has reached an `ACTIVE` state, continue on.

In [28]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

Deploying Started on:  02:00:31 AM
Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE
Deploying Completed on:  02:10:32 AM


## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting movie information to show for recommendations instead of just IDs.

In [29]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

USER: 472


In [31]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    return items.iloc[movie_id]['TITLE']

#### Select a User and an Item

#### Call GetRecommendations

The code below will get recommendations for the random user selected, it then places the recommendations in a dataframe and renders it.

In [32]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

Recommendations for user:  472


,Original Recommendations
0,Blade Runner (1982)
1,Go Fish (1994)
2,"White Balloon, The (1995)"
3,Father of the Bride Part II (1995)
4,Congo (1995)
5,Little Odessa (1994)
6,Sleepers (1996)
7,"Misérables, Les (1995)"
8,Jane Eyre (1996)
9,"Wild Bunch, The (1969)"


## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [33]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:059124553121:event-tracker/2a0e0f0d
d321b1b2-bdf0-4ee2-bb99-af19555e2ccc


In [34]:
event_tracker_arn = response['eventTrackerArn']

## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [35]:
session_dict = {}

In [36]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

The cell below will execute interactions and update the dataframe with the results so you can see the overall impact. Feel free to run the cell several times with movie IDs less than 1000. Note for this demo it will error if you use the same movie twice, this is due to how we showcase the headers.

In [38]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

,Original Recommendations,Return of the Jedi (1983),Delicatessen (1991)
0,Blade Runner (1982),Go Fish (1994),Raiders of the Lost Ark (1981)
1,Go Fish (1994),"Wild Bunch, The (1969)",Ridicule (1996)
2,"White Balloon, The (1995)",Fargo (1996),Bram Stoker's Dracula (1992)
3,Father of the Bride Part II (1995),Terminator 2: Judgment Day (1991),Blade Runner (1982)
4,Congo (1995),Blade Runner (1982),Weekend at Bernie's (1989)
5,Little Odessa (1994),Robert A. Heinlein's The Puppet Masters (1994),Fargo (1996)
6,Sleepers (1996),"Unbearable Lightness of Being, The (1988)","Godfather: Part II, The (1974)"
7,"Misérables, Les (1995)","Ghost and the Darkness, The (1996)",Mighty Aphrodite (1995)
8,Jane Eyre (1996),Bio-Dome (1996),M*A*S*H (1970)
9,"Wild Bunch, The (1969)",Bram Stoker's Dracula (1992),Dirty Dancing (1987)


## Conclusion

In this workshop you successfully created a dataset group, a dataset based on interactions, trained a recommendation model based on the data, deployed a campaign to generate recommendations, evaluated the initial recommendations, leveraged real time event tracking for even better recommendations, and below is bonus material to show how to export recommendations in  

This content will stay public on GitHub and can be used within your organization in order to build custom recommendation models, take a look at the various other notebooks in the future.

Thank you, and good luck in your next Machine Learning project!

BONUS MATERIAL BELOW:

## Batch Recommendations

So far you have seen how to generate recommendations via an API call and interact with event trackers. That works well for many applications but you may find yourself wanting to cache all recommendations for users locally or even to study the recommendations for new ideas. To support that Amazon Personalize supports batch exporting of your recommendations to a file as well. The cells below will walk you through sending recommendations to a file in S3 and then will show its contents. The file can be downloaded from S3 to your local computer as well.

First you will need to create a JSON file of user IDS

In [39]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

Now upload this file to S3:

In [40]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://059124553121reinventpersonalizeworkshop/json_input.json


Now that the input file is available in S3, you need to define where the output will go, and create a batch inference job.

In [41]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://059124553121reinventpersonalizeworkshop/


The cell below creates the batch job.

In [43]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

The next cell will poll until the export has completed. This is the final waiting loop of the workshop! This will again poll until it reaches an `ACTIVE` state, you can continue on once this is reached.

In [44]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  02:28:02 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

With the data successfully exported, grab the file and parse it.

In [46]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 561,User: 233,User: 579
0,George of the Jungle (1997),Ulee's Gold (1997),Star Wars (1977)
1,Shiloh (1997),Heat (1995),Georgia (1995)
2,Evita (1996),U Turn (1997),Bram Stoker's Dracula (1992)
3,L.A. Confidential (1997),Gattaca (1997),"Man of No Importance, A (1994)"
4,Gattaca (1997),Critical Care (1997),Blade Runner (1982)
5,U Turn (1997),"Wings of the Dove, The (1997)",Maverick (1994)
6,Contact (1997),Supercop (1992),Muriel's Wedding (1994)
7,Ulee's Gold (1997),FairyTale: A True Story (1997),Jaws 2 (1978)
8,Marvin's Room (1996),Breakdown (1997),Little Odessa (1994)
9,Hoodlum (1997),George of the Jungle (1997),Go Fish (1994)


In the cell above you can see the various recommendations for the users provided, in a real scenario the list of users could be your entire userbase allowing you to quickly reference and compare results between them.